In [ ]:
import json
import numpy as np
import pickle
import faiss
#78404883 4151 200

In [ ]:
with open('id_to_index.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    id_to_index = pickle.load(f)
print('loaded')

In [ ]:
with open('index_to_entitity.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    index_to_entitity = pickle.load(f)

In [ ]:
with open('index_to_relation.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    index_to_relation = pickle.load(f)

In [ ]:
index_to_relation[5002]

In [ ]:
all_embeddings = np.load("wikidata_translation_v1_vectors.npy", mmap_mode = 'r') #, skiprows=1, max_rows=78404883, usecols=range(1, 201), comments=None)

In [ ]:
class Constants:
    ENTITIES = all_embeddings[0:78404883]
    RELATIONS = all_embeddings[78404883:]
    

In [ ]:
entities_index = faiss.IndexFlatL2(200)
entities_index.add(Constants.ENTITIES)

In [ ]:
def closest_entitity(keyid, k, index, id_to_index, index_to_entitity):
    """
    key_id: a string representing an entitiy; ex: '<http://www.wikidata.org/entity/Q37079688>' 
    k: an int that indicates how many closest neighbors to search for
    id_to_index: dictionary that maps id to index
    index_to_id: dictionary that maps index to id
    """
    result = []

    vector = np.array([Constants.ENTITIES[id_to_index[keyid]]])
    
    _, neighbors = index.search(vector, k)
    
    for i in range(k):
        temp = index_to_entitity[neighbors[0,i]]
        result.append(temp)
    
    return result
    

    

In [ ]:
#This can be changed to whatever entity you want to perform the search on
key_id = '<http://www.wikidata.org/entity/Q76>'

In [ ]:
#These parameters may be changed based on the comments above within the function
print(closest_entitity(key_id,5,entities_index,id_to_index,index_to_entitity))

In [ ]:
relations_index = faiss.IndexFlatL2(200)
relations_index.add(Constants.RELATIONS)

In [ ]:
def closest_relation(key1, key2, k, index, id_to_index, index_to_relation):
    """
    key_id: a string representing an entitiy; ex: '<http://www.wikidata.org/entity/Q37079688>' 
    k: an int that indicates how many closest neighbors to search for
    id_to_index: dictionary that maps id to index
    index_to_id: dictionary that maps index to id
    """
    result = []

    vector1 = np.array([Constants.ENTITIES[id_to_index[key1]]])
    vector2 = np.array([Constants.ENTITIES[id_to_index[key2]]])
    
    
    
    vector = vector1 - vector2
    
    _, neighbors = index.search(vector, k)
    
    for i in range(k):
        temp = index_to_relation[neighbors[0,i]]
        result.append(temp)
    
    return result
    

In [ ]:
def dot_relation(key1, key2, k, index, id_to_index, index_to_relation):
    """
    key_id: a string representing an entitiy; ex: '<http://www.wikidata.org/entity/Q37079688>' 
    k: an int that indicates how many closest neighbors to search for
    id_to_index: dictionary that maps id to index
    index_to_id: dictionary that maps index to id
    """
    result = []

    vector1 = np.array([Constants.ENTITIES[id_to_index[key1]]])
    vector2 = np.array([Constants.ENTITIES[id_to_index[key2]]])
    
    scores = (vector1.reshape((1, 200)) @ (Constants.RELATIONS +vector2).transpose())/(np.linalg.norm(vector1)*np.linalg.norm(Constants.RELATIONS +vector2))
    
    result = []
    
    new = np.argsort(scores.flatten())
    for elt in new[-k:]:
        result.append(index_to_relation[elt])
    return result
    

In [ ]:
key1 = '<http://www.wikidata.org/entity/Q76>'
key2 = '<http://www.wikidata.org/entity/Q13133>'

In [ ]:
print(closest_relation(key1,key2,5,relations_index,id_to_index,index_to_relation))

In [ ]:
print(dot_relation(key1,key2,5,relations_index,id_to_index,index_to_relation))

In [ ]:
print(dot_relation(key2,key1,5,relations_index,id_to_index,index_to_relation))